# Settings Schemas

> JSON schemas and utilities for workflow settings

In [ ]:
#| default_exp settings.schemas

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass, field
from typing import Dict, Any, List, Optional, ClassVar

from fastcore.basics import patch

from cjm_fasthtml_jsonschema.core.dataclass import (
    SCHEMA_TITLE, SCHEMA_DESC, SCHEMA_MIN, SCHEMA_MAX, SCHEMA_ENUM,
    SCHEMA_MIN_LEN, SCHEMA_MAX_LEN, SCHEMA_PATTERN, SCHEMA_FORMAT,
    dataclass_to_jsonschema
)
from cjm_fasthtml_file_browser.core.config import BrowserConfig

from cjm_fasthtml_workflow_transcription_single_file.core.config import SingleFileWorkflowConfig
from cjm_fasthtml_workflow_transcription_single_file.storage.config import StorageConfig

## WorkflowSettings

User-configurable settings for the single-file transcription workflow. This dataclass serves as the single source of truth for settings that:

1. Are exposed in the settings UI
2. Get persisted to storage (currently JSON files, future: database)
3. Can be modified by users at runtime

The field metadata is used to auto-generate JSON schemas for form UI generation.

In [ ]:
#| export
@dataclass
class WorkflowSettings:
    """User-configurable settings for single-file transcription workflow."""
    
    # Class-level schema metadata (ClassVar excludes these from dataclass fields)
    __schema_name__: ClassVar[str] = "single_file_workflow"
    __schema_title__: ClassVar[str] = "Single File Transcription Settings"
    __schema_description__: ClassVar[str] = "Configure file scanning, storage, and workflow behavior"
    
    # File browser settings (field names match BrowserConfig for seamless config loading)
    directories: List[str] = field(
        default_factory=list,
        metadata={
            SCHEMA_TITLE: "Media Directories",
            SCHEMA_DESC: "Directories to scan for media files"
        }
    )
    enabled_types: List[str] = field(
        default_factory=lambda: ["audio", "video"],
        metadata={
            SCHEMA_TITLE: "Enabled File Types",
            SCHEMA_DESC: "File types to include in scan (audio, video)"
        }
    )
    recursive_scan: bool = field(
        default=True,
        metadata={
            SCHEMA_TITLE: "Recursive Scan",
            SCHEMA_DESC: "Scan subdirectories"
        }
    )
    items_per_page: int = field(
        default=30,
        metadata={
            SCHEMA_TITLE: "Items Per Page",
            SCHEMA_DESC: "Number of files to show per page",
            SCHEMA_MIN: 10,
            SCHEMA_MAX: 100
        }
    )
    default_view: str = field(
        default="list",
        metadata={
            SCHEMA_TITLE: "Default View",
            SCHEMA_DESC: "Default view mode for file selection",
            SCHEMA_ENUM: ["list"]  # Currently only list view is implemented
        }
    )
    
    # Storage settings
    auto_save: bool = field(
        default=True,
        metadata={
            SCHEMA_TITLE: "Auto-save Results",
            SCHEMA_DESC: "Automatically save transcription results when complete"
        }
    )
    results_directory: str = field(
        default="transcription_results",
        metadata={
            SCHEMA_TITLE: "Results Directory",
            SCHEMA_DESC: "Directory to save transcription results"
        }
    )
    
    # Resource management settings
    gpu_memory_threshold_percent: float = field(
        default=45.0,
        metadata={
            SCHEMA_TITLE: "GPU Memory Threshold (%)",
            SCHEMA_DESC: "GPU memory usage threshold for conflict detection (0-100)",
            SCHEMA_MIN: 0,
            SCHEMA_MAX: 100
        }
    )

## WORKFLOW_SETTINGS_SCHEMA

Auto-generated JSON schema for workflow settings. Used by the settings modal to generate forms for configuring the workflow.

In [ ]:
# Test dataclass_to_jsonschema conversion
schema = dataclass_to_jsonschema(WorkflowSettings)
assert schema["name"] == "single_file_workflow"
assert schema["title"] == "Single File Transcription Settings"
assert "directories" in schema["properties"]
assert schema["properties"]["directories"]["type"] == "array"
assert schema["properties"]["directories"]["title"] == "Media Directories"  # UI still shows "Media Directories"
assert "enabled_types" in schema["properties"]
assert schema["properties"]["enabled_types"]["type"] == "array"
assert schema["properties"]["items_per_page"]["minimum"] == 10
assert schema["properties"]["items_per_page"]["maximum"] == 100
print("Schema name:", schema["name"])
print("Properties:", list(schema["properties"].keys()))

Schema name: single_file_workflow
Properties: ['directories', 'enabled_types', 'recursive_scan', 'items_per_page', 'default_view', 'auto_save', 'results_directory', 'gpu_memory_threshold_percent']


In [ ]:
#| export
WORKFLOW_SETTINGS_SCHEMA = dataclass_to_jsonschema(WorkflowSettings) # Auto-generate schema from WorkflowSettings dataclass

## WorkflowSettings Methods

Methods for converting between `WorkflowSettings` and the runtime config objects.

In [ ]:
#| export
@patch(cls_method=True)
def from_configs(
    cls: WorkflowSettings,
    browser_config: BrowserConfig,  # BrowserConfig instance with file browser settings
    storage_config: StorageConfig,  # StorageConfig instance with result storage settings
    workflow_config: Optional[SingleFileWorkflowConfig] = None  # Optional workflow config for additional settings
) -> "WorkflowSettings":  # WorkflowSettings instance with values from configs
    """Create WorkflowSettings from runtime config objects."""
    return cls(
        # File browser settings
        directories=browser_config.directories,
        enabled_types=browser_config.enabled_types,
        recursive_scan=browser_config.recursive_scan,
        items_per_page=browser_config.items_per_page,
        default_view=browser_config.default_view,
        # Storage settings
        auto_save=storage_config.auto_save,
        results_directory=storage_config.results_directory,
        # Workflow settings
        gpu_memory_threshold_percent=workflow_config.gpu_memory_threshold_percent if workflow_config else 45.0
    )

In [ ]:
#| export
@patch
def apply_to_configs(
    self: WorkflowSettings,
    browser_config: BrowserConfig,  # BrowserConfig instance to update
    storage_config: StorageConfig,  # StorageConfig instance to update
    workflow_config: Optional[SingleFileWorkflowConfig] = None  # Optional workflow config to update
) -> None:
    """Apply settings to runtime config objects."""
    # File browser settings
    browser_config.directories = self.directories
    browser_config.enabled_types = self.enabled_types
    browser_config.recursive_scan = self.recursive_scan
    browser_config.items_per_page = self.items_per_page
    browser_config.default_view = self.default_view
    
    # Storage settings
    storage_config.auto_save = self.auto_save
    storage_config.results_directory = self.results_directory
    
    # Workflow settings
    if workflow_config:
        workflow_config.gpu_memory_threshold_percent = self.gpu_memory_threshold_percent

In [ ]:
#| export
@patch
def to_dict(
    self: WorkflowSettings
) -> Dict[str, Any]:  # Dictionary of settings values
    """Convert settings to a dictionary for serialization."""
    return {
        "directories": self.directories,
        "enabled_types": self.enabled_types,
        "recursive_scan": self.recursive_scan,
        "items_per_page": self.items_per_page,
        "default_view": self.default_view,
        "auto_save": self.auto_save,
        "results_directory": self.results_directory,
        "gpu_memory_threshold_percent": self.gpu_memory_threshold_percent,
    }

In [ ]:
# Test from_configs and apply_to_configs
browser = BrowserConfig(directories=["/test/path"], enabled_types=["audio"])
storage = StorageConfig(auto_save=False, results_directory="custom_results")

settings = WorkflowSettings.from_configs(browser, storage)
assert settings.directories == ["/test/path"]
assert settings.enabled_types == ["audio"]
assert settings.auto_save == False
assert settings.results_directory == "custom_results"

# Test apply_to_configs
new_browser = BrowserConfig()
new_storage = StorageConfig()
settings.apply_to_configs(new_browser, new_storage)
assert new_browser.directories == ["/test/path"]
assert new_browser.enabled_types == ["audio"]
assert new_storage.auto_save == False

# Test to_dict uses correct field names for config loading
d = settings.to_dict()
assert "directories" in d  # Must match BrowserConfig field name
assert "enabled_types" in d
assert d["directories"] == ["/test/path"]
assert d["enabled_types"] == ["audio"]

print("from_configs, apply_to_configs, and to_dict work correctly")

from_configs, apply_to_configs, and to_dict work correctly


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()